# Lab 4 - Oral Flag Feedback

Most of your are probably familiar with Dr. Malone's oral flag presentation review website.  The file `OralFlag_example.csv` contains an example table dump from associated Dr. Malone's database.  Your job is to make an number of summary files, as described below.

## <font color="red"> Problem 1 - Read and Inspect the File</font>

Read the file `OralFlag_example.csv` and inspect the columns, looking for any possible errors.  Note that the file is missing a header, so I have provided the column labels below.

In [12]:
import pandas as pd
from dfply import *

In [2]:
oral_flag_columns = ['Id', 'Time', 'Term', 'Submission', 'Group', 'Reviewer',
                     'Knowledge_of_Subject', 'Clear_Outcomes', 'Organization', 'Delivery',
                     'Comments_Most_Effective', 'Comments_Improvements']

In [5]:
df = pd.read_csv("./data/OralFlag_example.csv",header=None,names=oral_flag_columns)

In [4]:
cleanup_nums = {"Comments_Most_Effective": {"Silas is dumb": "Silas is a dork", "Iverson >> Bergen": "Iverson > Bergen"},
                "Comments_Improvements": {"Silas is dumb": "Silas is a dork", "Iverson >> Bergen": "Iverson > Bergen"}}
df.replace(cleanup_nums, inplace=True)
df.head()

,Id,Time,Term,Submission,Group,Reviewer,Knowledge_of_Subject,Clear_Outcomes,Organization,Delivery,Comments_Most_Effective,Comments_Improvements
0,1,2011-01-01 00:00:00.000000000,Fall2018,1,1,Chris,3,2,1,3,I hate R almost as much as Excel!!1!,I hate R almost as much as Excel!!1!
1,2,2011-01-01 00:14:32.727272704,Fall2018,1,1,Brant,2,3,3,2,Silas looks young enough to be a student,Silas looks young enough to be a student
2,3,2011-01-01 00:29:05.454545408,Fall2018,1,1,April,3,1,3,1,I hate R almost as much as Excel!!1!,I hate R almost as much as Excel!!1!
3,4,2011-01-01 00:43:38.181818112,Fall2018,1,1,Tisha,2,1,2,3,Silas is a dork,Silas is a dork
4,5,2011-01-01 00:58:10.909090816,Fall2018,1,1,Todd,3,1,1,1,Iverson > Bergen,Iverson > Bergen


## <font color="red"> Problem 2 - Scoring Table</font>

First, you need to make a scoring table, which provides an average of the scores for each group, along with the final score for that group.  The weighted score is the average of the instructor total score (assume the instructor is `"Todd"`) and the average total for the rest of the reviewers.

For example, consider the following scores.  The computation of the total score is illustrated below.  Make sure your final table has the following columns: `Group`, `Knowledge_of_Subject_mean`, `Clear_Outcomes_mean`,  `Organization_mean`, `Delivery_mean`, `overall_score`

**For full credit, do this in one pipe.**

In [6]:
# Example of how to compute the weighted score
scores = pd.DataFrame({'Reviewer':["Todd", "Chris", "Silas", "Tisha"],
                       'Knowledge_of_Subject':[2,3,4, 4],
                       'Clear_Outcomes':      [3,3,2, 4],  
                       'Organization':        [4,3,4, 4], 
                       'Delivery':            [2,3,3, 4]})

silas_score = 4+2+4+3
chris_score = 3+3+3+3
tisha_score = 4+4+4+4
todd_score = 2+3+4+2

overall_score = round(0.5*todd_score + 0.5*(silas_score + chris_score + tisha_score)/3, 2)
overall_score

12.33

In [21]:
df_head = df.head()

not_instructors = ["Chris","Brant","April","Tisha"]

(df_head 
 >> select(X.Submission, X.Group,X.Reviewer,X.Knowledge_of_Subject,X.Clear_Outcomes,X.Organization,X.Delivery)
 >> gather("category","score",columns_from('Knowledge_of_Subject'), add_id=True)
 >> group_by(X.Submission,X.Group)
)

,Submission,Group,Reviewer,_ID,category,score
0,1,1,Chris,0,Knowledge_of_Subject,3
1,1,1,Brant,1,Knowledge_of_Subject,2
2,1,1,April,2,Knowledge_of_Subject,3
3,1,1,Tisha,3,Knowledge_of_Subject,2
4,1,1,Todd,4,Knowledge_of_Subject,3
5,1,1,Chris,0,Clear_Outcomes,2
6,1,1,Brant,1,Clear_Outcomes,3
7,1,1,April,2,Clear_Outcomes,1
8,1,1,Tisha,3,Clear_Outcomes,1
9,1,1,Todd,4,Clear_Outcomes,1


## <font color="red"> Problem 3 - Comment files</font>

Your final task is to make a comment data frame for each group and write each file to a csv.  Each file should have three columns: Group, Comments_Most_Effective, Comments_Needed_Improvements; with the second and third columns containing all the respective comments for that group. 

To complete this task, you should

1. Write a lambda function that takes the original table and a group number and returns a table containing the comments for that group.
2. Write a lambda function that takes the output from 1. and reshapes the data so the comments from the same reviewer are on the same line.
3. Write a lambda function that composes the last two function.
4. Write a for loop that constructs and writes out a table for each group.  Recall that you can write a Pandas `df` to a csv file using `df.to_csv('filename', index=False)`.